# High Revenue, Low Profit Analysis (Very Strong Signal)

## Objective
Identify products and sub-categories that generate high revenue but deliver low or negative profit.
These items represent a **very strong signal** for pricing, cost, or discount optimization opportunities.

## Analysis Steps
1. **Load Data**: Auto-detect CSV file in the data/ directory
2. **Column Mapping**: Flexibly detect and map product, sub-category, sales, and profit columns
3. **Aggregate**: Compute total sales, total profit, and order count by product and sub-category
4. **Flag High Revenue, Low Profit Items**: Apply configurable thresholds
   - High sales threshold: Top 20% by total sales
   - Low profit threshold: Bottom 20% by profit OR negative/zero profit
5. **Visualize**: Create scatter plots highlighting flagged items
6. **Export**: Save flagged results as CSV files for further investigation
7. **Recommendations**: Provide actionable next steps

## Why This Matters
High-revenue, low-profit products can significantly drag down overall profitability.
Identifying them allows the business to:
- Review and adjust pricing strategies
- Investigate excessive discounts
- Examine cost of goods sold (COGS) issues
- Consider discontinuing unprofitable items despite high sales volume

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from pathlib import Path

# Configure visualization settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Set pandas display options to show more columns
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

## Step 1: Load Data

We'll automatically detect CSV files in the data/ directory using glob.
This makes the notebook flexible and reusable across different data files.

In [ ]:
# Auto-detect CSV file in the data/ directory
data_dir = Path('../data')
csv_files = list(data_dir.glob('*.csv'))

if not csv_files:
    raise FileNotFoundError(
        "No CSV files found in the data/ directory. "
        "Please ensure there is at least one CSV file in the ../data/ folder."
    )

# Use the first CSV file found
csv_path = csv_files[0]
print(f"Loading data from: {csv_path}")

# Load the data
df = pd.read_csv(csv_path)

print(f"\nData loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nFirst few rows:")
df.head()

## Step 2: Column Detection and Mapping

We'll flexibly detect common column names for:
- **Product**: Product, Product Name
- **Sub-Category**: Sub-Category, Sub Category, SubCategory
- **Sales**: Sales
- **Profit**: Profit

This ensures the notebook works with various CSV formats.

In [ ]:
# Display available columns
print("Available columns in dataset:")
print(df.columns.tolist())
print()

# Flexible column mapping
def find_column(df, possible_names):
    """Find a column by checking multiple possible names (case-insensitive)."""
    for col in df.columns:
        for name in possible_names:
            if col.lower() == name.lower() or col.lower().replace(' ', '') == name.lower().replace(' ', ''):
                return col
    return None

# Detect column names
product_col = find_column(df, ['Product', 'Product Name', 'ProductName'])
subcategory_col = find_column(df, ['Sub-Category', 'Sub Category', 'SubCategory', 'Subcategory'])
sales_col = find_column(df, ['Sales'])
profit_col = find_column(df, ['Profit'])

# Report detected columns
print("Detected columns:")
print(f"  Product column: {product_col}")
print(f"  Sub-Category column: {subcategory_col}")
print(f"  Sales column: {sales_col}")
print(f"  Profit column: {profit_col}")

# Verify required columns exist
if not sales_col or not profit_col:
    raise ValueError(
        "Required columns 'Sales' and 'Profit' must be present in the dataset. "
        f"Found columns: {df.columns.tolist()}"
    )

## Step 3: Data Cleaning

Convert Sales and Profit to numeric values and drop rows with missing values in these critical columns.

In [ ]:
# Convert Sales and Profit to numeric, coercing errors to NaN
df[sales_col] = pd.to_numeric(df[sales_col], errors='coerce')
df[profit_col] = pd.to_numeric(df[profit_col], errors='coerce')

# Count missing values before cleaning
missing_before = df[[sales_col, profit_col]].isna().sum()
print("Missing values before cleaning:")
print(missing_before)

# Drop rows with missing Sales or Profit
rows_before = len(df)
df = df.dropna(subset=[sales_col, profit_col])
rows_after = len(df)

print(f"\nRows removed: {rows_before - rows_after:,}")
print(f"Rows remaining: {rows_after:,}")

# Basic statistics
print(f"\nSales and Profit statistics:")
print(df[[sales_col, profit_col]].describe())

## Step 4: Aggregation by Product and Sub-Category

We'll aggregate the data at two levels:
1. **Product level**: Total sales, total profit, and number of orders per product
2. **Sub-Category level**: Total sales, total profit, and number of orders per sub-category

This allows us to identify problematic items at different granularity levels.

In [ ]:
# Product-level aggregation (if Product column exists)
if product_col:
    print("Aggregating by Product...")
    product_agg = df.groupby(product_col).agg({
        sales_col: 'sum',
        profit_col: 'sum',
        product_col: 'count'  # Count number of orders
    }).rename(columns={
        sales_col: 'total_sales',
        profit_col: 'total_profit',
        product_col: 'n_orders'
    }).reset_index()
    
    # Sort by total_sales descending
    product_agg = product_agg.sort_values('total_sales', ascending=False)
    
    print(f"\nProduct aggregation complete: {len(product_agg):,} unique products")
    print(f"\nTop 10 products by sales:")
    print(product_agg.head(10))
else:
    print("Product column not found. Skipping product-level aggregation.")
    product_agg = None

In [ ]:
# Sub-Category-level aggregation (if Sub-Category column exists)
if subcategory_col:
    print("Aggregating by Sub-Category...")
    subcategory_agg = df.groupby(subcategory_col).agg({
        sales_col: 'sum',
        profit_col: 'sum',
        subcategory_col: 'count'  # Count number of orders
    }).rename(columns={
        sales_col: 'total_sales',
        profit_col: 'total_profit',
        subcategory_col: 'n_orders'
    }).reset_index()
    
    # Sort by total_sales descending
    subcategory_agg = subcategory_agg.sort_values('total_sales', ascending=False)
    
    print(f"\nSub-Category aggregation complete: {len(subcategory_agg):,} unique sub-categories")
    print(f"\nAll sub-categories by sales:")
    print(subcategory_agg)
else:
    print("Sub-Category column not found. Skipping sub-category-level aggregation.")
    subcategory_agg = None

## Step 5: Flagging Logic - Identify High Revenue, Low Profit Items

### Configurable Thresholds
- **High Sales Threshold**: Top 20% of items by total sales (configurable)
- **Low Profit Threshold**: Bottom 20% of items by profit (configurable)
- **Negative/Zero Profit**: Items with profit <= 0

### Flags Created
1. `high_sales_flag`: Item is in top 20% by sales
2. `low_or_negative_profit_flag`: Item has profit <= 0
3. `low_profit_bottom_pct_flag`: Item is in bottom 20% by profit
4. `flagged_strong`: **High sales AND (negative profit OR bottom 20% profit)**

The `flagged_strong` items are our primary targets for investigation.

In [ ]:
# Configurable thresholds (percentiles)
HIGH_SALES_PERCENTILE = 80  # Top 20%
LOW_PROFIT_PERCENTILE = 20  # Bottom 20%

print(f"Configuration:")
print(f"  High Sales Threshold: Top {100 - HIGH_SALES_PERCENTILE}% (above {HIGH_SALES_PERCENTILE}th percentile)")
print(f"  Low Profit Threshold: Bottom {LOW_PROFIT_PERCENTILE}% (below {LOW_PROFIT_PERCENTILE}th percentile)")

In [ ]:
# Apply flagging logic to products
if product_agg is not None:
    print("\nApplying flagging logic to products...")
    
    # Calculate thresholds
    high_sales_threshold = product_agg['total_sales'].quantile(HIGH_SALES_PERCENTILE / 100)
    low_profit_threshold = product_agg['total_profit'].quantile(LOW_PROFIT_PERCENTILE / 100)
    
    print(f"\nProduct thresholds:")
    print(f"  High sales threshold: ${high_sales_threshold:,.2f}")
    print(f"  Low profit threshold: ${low_profit_threshold:,.2f}")
    
    # Create flags
    product_agg['high_sales_flag'] = product_agg['total_sales'] >= high_sales_threshold
    product_agg['low_or_negative_profit_flag'] = product_agg['total_profit'] <= 0
    product_agg['low_profit_bottom_pct_flag'] = product_agg['total_profit'] <= low_profit_threshold
    
    # Strong signal: High sales AND (negative profit OR low profit)
    product_agg['flagged_strong'] = (
        product_agg['high_sales_flag'] & 
        (product_agg['low_or_negative_profit_flag'] | product_agg['low_profit_bottom_pct_flag'])
    )
    
    # Count flagged items
    n_flagged = product_agg['flagged_strong'].sum()
    print(f"\n⚠️  Products flagged as High Revenue, Low Profit: {n_flagged}")
    
    if n_flagged > 0:
        print(f"\nFlagged products:")
        flagged_products = product_agg[product_agg['flagged_strong']].copy()
        print(flagged_products[[product_col, 'total_sales', 'total_profit', 'n_orders']].to_string(index=False))
else:
    print("No product aggregation available.")
    flagged_products = None

In [ ]:
# Apply flagging logic to sub-categories
if subcategory_agg is not None:
    print("\nApplying flagging logic to sub-categories...")
    
    # Calculate thresholds
    high_sales_threshold = subcategory_agg['total_sales'].quantile(HIGH_SALES_PERCENTILE / 100)
    low_profit_threshold = subcategory_agg['total_profit'].quantile(LOW_PROFIT_PERCENTILE / 100)
    
    print(f"\nSub-Category thresholds:")
    print(f"  High sales threshold: ${high_sales_threshold:,.2f}")
    print(f"  Low profit threshold: ${low_profit_threshold:,.2f}")
    
    # Create flags
    subcategory_agg['high_sales_flag'] = subcategory_agg['total_sales'] >= high_sales_threshold
    subcategory_agg['low_or_negative_profit_flag'] = subcategory_agg['total_profit'] <= 0
    subcategory_agg['low_profit_bottom_pct_flag'] = subcategory_agg['total_profit'] <= low_profit_threshold
    
    # Strong signal: High sales AND (negative profit OR low profit)
    subcategory_agg['flagged_strong'] = (
        subcategory_agg['high_sales_flag'] & 
        (subcategory_agg['low_or_negative_profit_flag'] | subcategory_agg['low_profit_bottom_pct_flag'])
    )
    
    # Count flagged items
    n_flagged = subcategory_agg['flagged_strong'].sum()
    print(f"\n⚠️  Sub-Categories flagged as High Revenue, Low Profit: {n_flagged}")
    
    if n_flagged > 0:
        print(f"\nFlagged sub-categories:")
        flagged_subcategories = subcategory_agg[subcategory_agg['flagged_strong']].copy()
        print(flagged_subcategories[[subcategory_col, 'total_sales', 'total_profit', 'n_orders']].to_string(index=False))
else:
    print("No sub-category aggregation available.")
    flagged_subcategories = None

## Step 6: Visualizations

### Scatter Plot: Total Sales vs Total Profit

We'll create scatter plots to visualize the relationship between sales and profit,
with flagged items highlighted in red. This makes it easy to spot high-revenue, low-profit outliers.

In [ ]:
# Scatter plot for products
if product_agg is not None:
    plt.figure(figsize=(12, 7))
    
    # Plot non-flagged items in gray
    non_flagged = product_agg[~product_agg['flagged_strong']]
    plt.scatter(non_flagged['total_sales'], non_flagged['total_profit'], 
                alpha=0.5, s=50, c='gray', label='Other Products')
    
    # Plot flagged items in red
    flagged = product_agg[product_agg['flagged_strong']]
    plt.scatter(flagged['total_sales'], flagged['total_profit'], 
                alpha=0.8, s=100, c='red', edgecolors='darkred', linewidths=1.5,
                label='High Revenue, Low Profit ⚠️')
    
    # Annotate top flagged items
    if len(flagged) > 0:
        # Annotate up to 5 flagged items with highest sales
        top_flagged = flagged.nlargest(min(5, len(flagged)), 'total_sales')
        for idx, row in top_flagged.iterrows():
            # Truncate long product names
            name = row[product_col]
            if len(name) > 30:
                name = name[:27] + '...'
            plt.annotate(name, 
                        xy=(row['total_sales'], row['total_profit']),
                        xytext=(10, 10), textcoords='offset points',
                        fontsize=9, alpha=0.8,
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.6),
                        arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', alpha=0.6))
    
    # Add reference lines
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Breakeven')
    
    plt.xlabel('Total Sales ($)', fontsize=12, fontweight='bold')
    plt.ylabel('Total Profit ($)', fontsize=12, fontweight='bold')
    plt.title('Product Analysis: Total Sales vs Total Profit\n(High Revenue, Low Profit items highlighted)', 
              fontsize=14, fontweight='bold', pad=20)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nScatter plot shows {len(flagged)} flagged products in red.")

In [ ]:
# Scatter plot for sub-categories
if subcategory_agg is not None:
    plt.figure(figsize=(12, 7))
    
    # Plot non-flagged items in blue
    non_flagged = subcategory_agg[~subcategory_agg['flagged_strong']]
    plt.scatter(non_flagged['total_sales'], non_flagged['total_profit'], 
                alpha=0.6, s=150, c='lightblue', edgecolors='blue', linewidths=1,
                label='Other Sub-Categories')
    
    # Plot flagged items in red
    flagged = subcategory_agg[subcategory_agg['flagged_strong']]
    plt.scatter(flagged['total_sales'], flagged['total_profit'], 
                alpha=0.9, s=200, c='red', edgecolors='darkred', linewidths=2,
                label='High Revenue, Low Profit ⚠️')
    
    # Annotate ALL flagged sub-categories (there are typically few)
    if len(flagged) > 0:
        for idx, row in flagged.iterrows():
            plt.annotate(row[subcategory_col], 
                        xy=(row['total_sales'], row['total_profit']),
                        xytext=(10, 10), textcoords='offset points',
                        fontsize=11, fontweight='bold',
                        bbox=dict(boxstyle='round,pad=0.4', facecolor='yellow', alpha=0.7),
                        arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2', 
                                       lw=1.5, alpha=0.7))
    
    # Add reference lines
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Breakeven')
    
    plt.xlabel('Total Sales ($)', fontsize=12, fontweight='bold')
    plt.ylabel('Total Profit ($)', fontsize=12, fontweight='bold')
    plt.title('Sub-Category Analysis: Total Sales vs Total Profit\n(High Revenue, Low Profit items highlighted)', 
              fontsize=14, fontweight='bold', pad=20)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nScatter plot shows {len(flagged)} flagged sub-categories in red.")

In [ ]:
# Optional: Bar chart for flagged products
if product_agg is not None and 'flagged_products' in locals() and len(flagged_products) > 0:
    print("\nCreating bar chart for flagged products...")
    
    # Limit to top 15 by sales for readability
    top_flagged = flagged_products.nlargest(15, 'total_sales')
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Sales bar chart
    ax1.barh(range(len(top_flagged)), top_flagged['total_sales'], color='steelblue')
    ax1.set_yticks(range(len(top_flagged)))
    # Truncate labels for readability
    labels = [name[:40] + '...' if len(name) > 40 else name 
              for name in top_flagged[product_col]]
    ax1.set_yticklabels(labels, fontsize=9)
    ax1.set_xlabel('Total Sales ($)', fontsize=11, fontweight='bold')
    ax1.set_title('Flagged Products: Total Sales', fontsize=12, fontweight='bold')
    ax1.invert_yaxis()
    ax1.grid(axis='x', alpha=0.3)
    
    # Profit bar chart
    colors = ['red' if p < 0 else 'orange' for p in top_flagged['total_profit']]
    ax2.barh(range(len(top_flagged)), top_flagged['total_profit'], color=colors)
    ax2.set_yticks(range(len(top_flagged)))
    ax2.set_yticklabels(labels, fontsize=9)
    ax2.set_xlabel('Total Profit ($)', fontsize=11, fontweight='bold')
    ax2.set_title('Flagged Products: Total Profit', fontsize=12, fontweight='bold')
    ax2.axvline(x=0, color='black', linestyle='--', linewidth=1)
    ax2.invert_yaxis()
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Bar chart shows top {len(top_flagged)} flagged products.")

## Step 7: Export Flagged Items

Save flagged products and sub-categories to CSV files for further investigation and sharing with stakeholders.

In [ ]:
# Create output directory if it doesn't exist
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.absolute()}")

# Export flagged products
if product_agg is not None and 'flagged_products' in locals() and len(flagged_products) > 0:
    product_output_path = output_dir / 'flagged_products_high_revenue_low_profit.csv'
    flagged_products.to_csv(product_output_path, index=False)
    print(f"\n✅ Flagged products exported to: {product_output_path}")
    print(f"   Total products flagged: {len(flagged_products)}")
else:
    print("\nNo flagged products to export.")

# Export flagged sub-categories
if subcategory_agg is not None and 'flagged_subcategories' in locals() and len(flagged_subcategories) > 0:
    subcategory_output_path = output_dir / 'flagged_subcategories_high_revenue_low_profit.csv'
    flagged_subcategories.to_csv(subcategory_output_path, index=False)
    print(f"\n✅ Flagged sub-categories exported to: {subcategory_output_path}")
    print(f"   Total sub-categories flagged: {len(flagged_subcategories)}")
else:
    print("\nNo flagged sub-categories to export.")

print("\n✅ Export complete!")

## Step 8: Actionable Recommendations and Next Steps

### How to Interpret the Flags

**flagged_strong = True** means:
- The item is in the **top 20% by sales** (high revenue generator)
- AND the item has **negative/zero profit OR is in the bottom 20% by profit**

These items are **critical** because they:
1. Generate significant revenue (top performers)
2. But deliver poor profitability (dragging down margins)

### Recommended Next Actions

#### 1. Investigate Discounting Practices
- Pull discount data for flagged items
- Calculate average discount % for these products vs. non-flagged products
- **Action**: Consider reducing discount frequency or amount

#### 2. Review Cost of Goods Sold (COGS)
- Analyze supplier costs for flagged products
- Identify if COGS has increased recently
- **Action**: Negotiate better supplier terms or find alternative suppliers

#### 3. Adjust Pricing Strategy
- Calculate profit margin % (profit/sales) for flagged items
- Compare to industry benchmarks or similar products
- **Action**: Test price increases (e.g., 5-10%) and monitor demand elasticity

#### 4. Time-Based Analysis
- Segment flagged products by time period (e.g., monthly or quarterly)
- Identify if profitability issues are recent or long-term
- **Action**: Focus on items with declining profit trends

#### 5. Consider Product Discontinuation
- For items with consistently negative profit despite high sales
- Evaluate strategic importance (e.g., loss leaders)
- **Action**: Phase out non-strategic unprofitable products

#### 6. Calculate Profit Margin %
- Add a `profit_margin` column: `total_profit / total_sales * 100`
- Target improvement to at least industry-standard margins (typically 10-30% for retail)

### Next Steps for Analysis

1. **Join with discount data** to see if excessive discounts are driving low profits
2. **Time series analysis** to understand when profitability declined
3. **Customer segment analysis** to see which customer types buy these products
4. **Regional analysis** to identify if specific regions have profitability issues
5. **Competitive pricing research** to benchmark against market rates

### Key Metrics to Track Going Forward

- **Profit Margin %**: `(total_profit / total_sales) * 100`
- **Average Order Profit**: `total_profit / n_orders`
- **Sales Contribution %**: `(item_sales / total_sales) * 100`
- **Profit Contribution %**: `(item_profit / total_profit) * 100`

### Warning Signs to Monitor

- Items with profit margin < 10%
- Items with negative profit for 2+ consecutive quarters
- Items where sales are growing but profit is declining
- Items with high return rates (if data available)

## Conclusion

This analysis successfully identified products and sub-categories that generate high revenue but deliver low profit—a **very strong signal** for business optimization.

### Key Takeaways

1. **High-revenue, low-profit items are hidden drags on profitability**: They look successful on the sales chart but hurt the bottom line.

2. **Configurable thresholds enable flexible analysis**: By adjusting percentile cutoffs, we can focus on the most critical items.

3. **Multiple aggregation levels provide different perspectives**: Product-level analysis catches specific items, while sub-category analysis reveals systemic issues.

4. **Visualization makes patterns obvious**: Scatter plots quickly highlight outliers that need attention.

5. **Exported data enables action**: Stakeholders can use the CSV files to investigate further and implement changes.

### Impact Potential

Addressing even a handful of high-revenue, low-profit products can significantly improve overall profitability without sacrificing revenue. This is often a **quick win** for data-driven businesses.

---

**Next Steps**: Review the exported CSV files and begin investigating the flagged items using the recommended actions above.